# Livrable final du projet - Projet Green Graph
*Equipe CesiCDP - Chef de projet : Leila | Opérateurs : Tom, Edwin*</br>
## Sommaire :
    1. Introduction

    PARTIE 1 : Modélisation
    2. Rappel de la modélisation formelle
        2.1. Résumé des hypothèses et représentations
        2.2. Justification du maintien de la modélisation
    3. Méthodes de résolution
        3.1. Choix des algorithmes
        3.2. Description des algorithmes 
        3.3. Complexité des algorithmes 

    PARTIE 2 : Implémentation et exploitation
    4. Implémentation
        4.1. Détails de l’implémentation des algorithmes
        4.2. Cas de test représentatifs
        4.3. Résultats observés sur les tests
    5. Étude expérimentale
        5.1. Méthodologie du plan d’expérience
        5.2. Analyse des performances 
        5.3. Limites observées
        5.4. Perspectives d’amélioration

    6. Plan de Travail et Organisation du Projet
        6.1. Étapes prévues 

    7. Conclusion

    8. Annexes
        8.1. Glossaire
        8.2. Références bibliographiques
        8.3. Annexes techniques 

### 1. Introduction
Dans la continuité de notre premier livrable, ce document final a pour objectif de présenter l'ensemble de la démarche menée autour du projet Green Graph, de la modélisation initiale à l'évaluation expérimentale des solutions implémentées.

Pour rappel, on se questionnait sur comment réduire l’impact environnemental des tournées de livraison tout en optimisant les coûts opérationnels.<br>
En modélisant ce problème à partir du **Problème du Voyageur de Commerce (PVC)** complété avec des contraintes réalistes :
* **Coût ou restriction de passage sur certaines arêtes :** Certaines routes peuvent être plus coûteuses ou interdites,
* **Dépendances entre visites :** Une ville ne peut être visitée qu'après en avoir visité une autre,

nous avons pu définir une approche formelle qui répond aux besoins de planification d'un trajet routier réel.

La première partie du projet consiste à modéliser mathématiquement le problème, nous y détaillons également les méthodes de résolution sélectionnées, en mettant en lumière les raisons de nos choix algorithmiques, ainsi que la complexité associée.

La seconde partie est dédiée à la mise en œuvre : l’implémentation des algorithmes, les cas de test choisis, et l’analyse des résultats obtenus. Cette étude expérimentale permet d'évaluer la performance des solutions proposées, leurs limitations, et d’ouvrir la voie à de nouvelles perspectives d’amélioration.


## Partie 1
### 2. Rappel de la modélisation formelle
##### 2.1. Résumé des hypothèses et représentations
Pour rappel, notre réseau routié est représenté sous la forme d'un graphe  non orienté $G = (V, E)$, où :
* $V = {v_0, v_1, ..., v_n}$ représente les sommets (le dépôt $v_0$ et les points de livraison),
* $E$ l’ensemble des arêtes, représentant les routes entre les sommets,
* $E_{interdit}$ l'ensemble des routes inaccessible,
* $D$ l'ensemble des dépendances entre les villes.

Chaque arête $(u, v) \in E$ est associée à un coût $c_{uv}$ (distance), une variable binaire $x_{uv} \in \{0, 1\}$ indique si l’arête $(u, v)$ est utilisée dans la tournée. Si l'arête $x_{uv} \in E_{interdit}$, alors $x_{uv}=0$.

Chaque élément $d \in D$ est une paire ordonnée $(v_i, v_j)$ signifiant le sommet $v_i$ doit être visité avant le sommet $v_j$.<br>
On note $t_i \in \mathbb{N}$, une variable qui représente l’ordre de visite du sommet $v_i$ dans la tournée, pour chaque dépendance $(v_i, v_j) \in D$ : $t_i<t_j$.

On souligne qu'on ne visite chaque sommet $V$ de $G$ exactement une fois et on retourne au point de départ $v_0$, l'objectif est de miniser le coûts total de la tournée en respectant les contraintes soit $\min \sum_{(u,v) \in E} c_{uv} \cdot x_{uv}$.


In [15]:
# Installation des bibliothèques
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [16]:
from geopy.geocoders import Nominatim

#Par cette fonction, on convertit le nom des villes en coordonnées GPS (latitude, longitude). 
def geocode_city(city_name):
    geolocator = Nominatim(user_agent="routing_app", timeout=10)
    location = geolocator.geocode(city_name,country_codes="FR")
    
    if location:
        return (location.latitude, location.longitude)
    else:
        return None, None

In [17]:
import pandas as pd
import random
from tqdm import tqdm

"""
Cette fonction lit un CSV de noms de villes, en choisit aléatoirement un nombre donné, 
récupère leurs coordonnées via geocode_city et renvoie un dictionnaire city_map,
associant chaque ville à ses (latitude, longitude).
"""
def GenerateCityMapFromCSV(size):
    city_map = {} 
    df = pd.read_csv("CityName.csv", on_bad_lines='skip')
    city_list = df['City'].dropna().tolist()
    city_list = random.sample(city_list, size)
    for city in tqdm(city_list):
        lat, lon = geocode_city(city) 
        if  (lat or lon) is not None:
            city_map[city] = (lat,lon)
    return city_map

Cities = GenerateCityMapFromCSV(6)

100%|██████████| 6/6 [00:05<00:00,  1.16it/s]


In [18]:
import requests

def calculate_travel_time(departure_city, arrival_city, city_map, mode, osrm_link, params):
    """
    Calcule le temps de trajet et la distance entre deux villes en utilisant l'API OSRM.
    
    Args:
        departure_city (str): nom de la ville de départ.
        arrival_city (str): nom de la ville d'arrivée.
        city_map (dict): dictionnaire mappant les noms de villes à leurs coordonnées (lat, lon).
        mode (str): mode de transport ('driving', 'cycling', 'walking').
        osrm_link (str): URL de base de l'API OSRM.
        params (dict): paramètres additionnels pour la requête API.
    
    Retourne:
        tuple: (duration_seconds, distance_meters, formatted_time) où:
            - duration_seconds (float): durée du trajet en secondes,
            - distance_meters (float): distance parcourue en mètres,
            - formatted_time (str): durée formatée pour affichage (par ex. '1 hour 23 minutes').
    """
    # Récupération des coordonnées GPS des villes de départ et d'arrivée
    try:
        lat1, lon1 = city_map[departure_city]
        lat2, lon2 = city_map[arrival_city]
    except KeyError as e:
        # Si la ville n'existe pas dans city_map, on renvoie une erreur
        return (None, None, f"Ville introuvable: {e}")

    # Construction de l'URL pour l'appel à l'API OSRM
    url = f"{osrm_link}/{mode}/{lon1},{lat1};{lon2},{lat2}"

    # Appel HTTP GET à l'API
    response = requests.get(url, params=params, timeout=10)

    # Vérification du code HTTP retourné
    if response.status_code != 200:
        return (None, None, f"Erreur API (statut): {response.status_code}")

    data = response.json()

    # Vérification du code de réponse de l'API OSRM
    if data.get("code") != "Ok":
        return (None, None, f"Erreur OSRM: {data.get('code')}")

    # Extraction des informations de durée et de distance
    route = data["routes"][0]
    duration_seconds = route["duration"]
    distance_meters = route["distance"]

    # Formatage du temps en heures et minutes pour affichage
    hours, remainder = divmod(duration_seconds, 3600)
    minutes, _ = divmod(remainder, 60)

    formatted_time = ""
    if hours > 0:
        formatted_time += f"{int(hours)} heure{'s' if hours > 1 else ''} "
    if minutes > 0:
        formatted_time += f"{int(minutes)} minute{'s' if minutes > 1 else ''}"

    return (duration_seconds, distance_meters, formatted_time.strip())

In [19]:

def display_route(departure_city, arrival_city, city_map, mode, osrm_link, params):
    """
    Affiche les informations de trajet entre deux villes.
    
    Args :
        departure_city (str) : nom de la ville de départ
        arrival_city   (str) : nom de la ville d'arrivée
        city_map       (dict) : dictionnaire {ville: (lat, lon)}
        mode           (str) : mode de déplacement ("driving", "cycling", "walking")
        osrm_link      (str) : URL de base de l'API OSRM
        params         (dict): paramètres optionnels pour l'appel API
    """

    # Dictionnaire pour traduire le mode en phrase plus lisible
    modes = {
        "driving": "en voiture",
        "cycling": "à vélo",
        "walking": "à pied"
    }
    
    # Appel à la fonction qui interroge l'API OSRM pour obtenir durée et distance
    duration, distance, message = calculate_travel_time(departure_city, arrival_city, city_map, mode, osrm_link, params)
    
    if duration is None:
        print(message)
        return
    
    # Affichage des informations de trajet
    print(f"Route from {departure_city} to {arrival_city} {modes.get(mode, '')}:")
    print(f"Travel time: {message}")
    print(f"Distance: {distance/1000:.1f} km")

In [20]:
import pytz
from datetime import datetime

"""
Cette fonction construit une matrice carrée donnant, pour chaque paire de villes, 
le temps de trajet (en secondes) et la distance (en kilomètres)
"""

def matrix_generation(cities, mode, link, params, toPrint=False):
    matrix = []  # Matrice qui contiendra les sous-listes pour chaque ville source

    # On parcourt chaque ville comme point de départ
    for sourceCity in tqdm(cities):
        submatrix = []  # Sous-liste correspondant aux trajets depuis sourceCity

        # On parcourt chaque ville comme point d'arrivée
        for destinationCity in cities:
            if sourceCity is not destinationCity:
                # On appelle la fonction calculate_travel_time pour obtenir (durée, distance, message)
                duration, distance, _ = calculate_travel_time(
                    sourceCity, destinationCity, cities, mode, link, params
                )

                if toPrint:
                    # Si on souhaite un format "imprimable" :
                    # – Conversion de la durée (en secondes) en chaîne 'HH:MM:SS'
                    # – Conversion de la distance (en mètres) en kilomètres (entier)
                    submatrix.append([
                        datetime.fromtimestamp(duration, tz=pytz.utc).strftime('%H:%M:%S'),
                        int(distance / 1000)
                    ])
                else:
                    # Sinon on garde les valeurs brutes : durée (s) et distance (km entier)
                    submatrix.append([
                        duration,
                        int(distance / 1000)
                    ])
            else:
                # Pour la même ville (source == destination), pas de trajet : on met (0, 0)
                submatrix.append([0, 0])

        # On ajoute la sous-liste (pour sourceCity) à la matrice principale
        matrix.append(submatrix)

    # On renvoie la matrice complète : chaque ligne i correspond aux trajets de la ville i vers toutes les autres
    return matrix



##### 2.2. Justification du maintien de la modélisation
Après vérification, aucune modification n’a donc été nécessaire dans la structure mathématique pour répondre aux objectifs du projet. Nous avons décidé de conserver la structure et les représentations formelles définies dans le livrable précédent.

Cette modélisation s'est révélée suffisamment expressive pour prendre en compte l'ensemble des contraintes que nous souhaitons intégrer au problème.<br>
De plus, notre problème est basé sur le **problème du voyageur du commerce (TSP)**, bien que nous avons démontré que ce problème est difficile (NP-complet), il est applicable à d'autres problèmes de logistique.


### 3. Méthodes de résolution
##### 3.1. Choix des algorithmes
##### 3.2. Description des algorithmes 
##### 3.3. Complexité des algorithmes 



### 4. Implémentation
##### 4.1. Détails de l’implémentation des algorithmes
##### 4.2. Cas de test représentatifs
##### 4.3. Résultats observés sur les tests


### 5. Étude expérimentale
##### 5.1. Méthodologie du plan d’expérience
##### 5.2. Analyse des performances 
##### 5.3. Limites observées
##### 5.4. Perspectives d’amélioration



### 6. Plan de Travail et Organisation du Projet
##### 6.1. Étapes prévues 


### 7. Conclusion


### 8. Annexes
##### 8.1. Glossaire
##### 8.2. Références bibliographiques
##### 8.3. Annexes techniques 